# Entity Resolution 

Entity Resolution refers to the task of finding records in a dataset that refer to the same entity across different data sources. It is referred in different names such as "Record linkage" or "DeDuplication", a process of joining records from one data source with another that describe the same entity. 

ER is necessary when joining datasets based on entities that may or may not share a common identifier (e.g., database key, URI, National identification number), as may be the case due to differences in record shape, storage location, and/or curator style or preference. 

In [177]:
# Importing relevant libraries

import numpy as np
import pandas as pd
import recordlinkage
from recordlinkage.preprocessing import clean

In [ ]:
### Uploading the csv file as a dataframe

df1=pd.read_csv('Scholar.csv', index_col='row_id', engine ='python')

df2=pd.read_csv('DBLP.csv', index_col='row_id', engine ='python')


### Understanding the dataset

In [179]:
# Checking the shape of the dataset

print(df1.shape)
print(df2.shape)

(64260, 5)
(2616, 5)


In [ ]:
# Checking for null values across all the columns in the two datasets

print(df1.isnull().sum())

print('\n**********************\n')

print(df2.isnull().sum())

idScholar        0
title            0
authors          1
venue        14997
year         34789
dtype: int64

**********************

idDBLP       0
title        0
authors    218
venue      211
year         0
dtype: int64


Comment: There are no null values in the title column across both the datasets and I will use 'title' for indexing in the record linkage process. 

In [ ]:
# Renaming the index columns to match the output file format. 

df1.index.names = ['Scholar_Match']

df2.index.names = ['DBLP_Match']

In [ ]:
# Basic structure of the dataset (top 5)

df1.head()

,idScholar,title,authors,venue,year
Scholar_Match,,,,,
1,aKcZKwvwbQwJ,11578 Sorrento Valley Road,QD Inc,"San Diego,",NaN
2,ixKfiTHoaDoJ,Initiation of crazes in polystyrene,"AS Argon, JG Hannoosh","Phil. Mag,",NaN
3,3BxllB4wwcIJ,Immunogold labelling is a quantitative method ...,"GH Hansen, LL Wetterberg, H Sj͗�_str͗�_m, O No...","The Histochemical Journal,",1992.0
4,d2WWxwKMex4J,The Burden of Infectious Disease Among Inmates...,"TM Hammett, P Harmon, W Rhodes",see,NaN
5,cZCX-AQpjccJ,The Role of Faculty Advising in Science and En...,JR Cogdell,"NEW DIRECTIONS FOR TEACHING AND LEARNING,",1995.0


In [ ]:
df2.head()

,idDBLP,title,authors,venue,year
DBLP_Match,,,,,
1,conf/vldb/RusinkiewiczKTWM95,Towards a Cooperative Transaction Model - The ...,"M Rusinkiewicz, W Klas, T Tesch, J W_sch, P Muth",VLDB,1995
2,journals/sigmod/EisenbergM02,SQL/XML is Making Good Progress,"A Eisenberg, J Melton",SIGMOD Record,2002
3,conf/vldb/AmmannJR95,Using Formal Methods to Reason about Semantics...,"P Ammann, S Jajodia, I Ray",VLDB,1995
4,journals/sigmod/Liu02,Editor's Notes,L Liu,SIGMOD Record,2002
5,journals/sigmod/Hammer02,Report on the ACM Fourth International Worksho...,NaN,NaN,2002


### Data Preprocessing 

1. Pre-processing of data (cleaning and standardising) increases accuracy of entity resolution (or record linkage) process. 

2. In this section, clean () function from Python Record Linkage library is used for data cleaning. 


In [ ]:
# Cleaning the title, authors and venue columns in df1 (Scholar dataset)

df1['title']= clean(df1['title'])

df1['authors']= clean(df1['authors'])

df1['venue']= clean(df1['venue'])

Comment: The clean function from Python Record Linkage toolkit converts the text into lower case, replaces special characters and removes brackets

In [ ]:
# The dataset 1 (scholar.csv) after data cleaning

df1.head()

,idScholar,title,authors,venue,year
Scholar_Match,,,,,
1,aKcZKwvwbQwJ,11578 sorrento valley road,qd inc,san diego,NaN
2,ixKfiTHoaDoJ,initiation of crazes in polystyrene,as argon jg hannoosh,phil mag,NaN
3,3BxllB4wwcIJ,immunogold labelling is a quantitative method ...,gh hansen ll wetterberg h sj str m o norn,the histochemical journal,1992.0
4,d2WWxwKMex4J,the burden of infectious disease among inmates...,tm hammett p harmon w rhodes,see,NaN
5,cZCX-AQpjccJ,the role of faculty advising in science and en...,jr cogdell,new directions for teaching and learning,1995.0


In [ ]:
# Cleaning the title, authors and venue columns in df2 (DBLP dataset)

df2['title']= clean(df2['title'])

df2['authors']= clean(df2['authors'])

df2['venue']= clean(df2['venue'])

In [ ]:
# The dataset 2 (DBLP.csv) after data cleaning

df2.head()

,idDBLP,title,authors,venue,year
DBLP_Match,,,,,
1,conf/vldb/RusinkiewiczKTWM95,towards a cooperative transaction model the co...,m rusinkiewicz w klas t tesch j w sch p muth,vldb,1995
2,journals/sigmod/EisenbergM02,sqlxml is making good progress,a eisenberg j melton,sigmod record,2002
3,conf/vldb/AmmannJR95,using formal methods to reason about semantics...,p ammann s jajodia i ray,vldb,1995
4,journals/sigmod/Liu02,editors notes,l liu,sigmod record,2002
5,journals/sigmod/Hammer02,report on the acm fourth international worksho...,NaN,NaN,2002


### Step 1: Indexing

In [ ]:
# Indexing the datasets using sorted neighbourhood algorithm

indexer = recordlinkage.Index()
indexer.sortedneighbourhood(left_on='title', right_on='title')

candidates = indexer.index(df1, df2)
print(len(candidates))


9457


Comment:

1. Indexing creates pair of records called candidate links or candidate matches based on indexing algorithms such as 'full indexing', 'blocking' and 'sorted neighborhood'. 

2. Indexing links and detects duplicate records across the datasets and returns a complete set of record pairs. 

3. 'Sorted neighbourhood' indexing is used on 'title' column on both datasets resulting in records pairs of 9457 with default windowsize=3. A large window size results in more record pairs. 

4. The Sorted Neighbourhood Index method is a great method when there is relatively large amount of spelling mistakes. Blocking index will fail in that situation because it excludes to many records on minor spelling mistakes.

5. Full indexing leading to 16.5 million record pairs and it is computationally expensive and time-taking for running full indexing algorithm in the present context. 

###Step 2: Comparing 

1. It is used to compare records pairs as set of informative, discriminating and independent features is important for a good classification of record pairs into matching and distinct pairs. 

In [ ]:
# Comparing the record pairs across the chosen columns 'title', 'authors', 'venue' and 'year'

compare = recordlinkage.Compare()

compare.string('title', 'title', threshold=0.85, label='title')
compare.string('authors',
            'authors',
            method= 'jarowinkler',
            threshold=0.85,
            label='authors')
compare.string('venue',
            'venue',
            threshold=0.85,
            label='venue')
compare.numeric('year',
            'year',
            label='year')
features = compare.compute(candidates, df1, df2)

In [ ]:
features

,,title,authors,venue,year
Scholar_Match,DBLP_Match,,,,
35,2457,0.0,0.0,0.0,0.0
54,2103,0.0,0.0,0.0,0.0
89,1811,0.0,0.0,0.0,0.0
170,1691,0.0,0.0,0.0,0.0
192,841,0.0,0.0,0.0,0.0
...,...,...,...,...,...
64144,1103,0.0,0.0,0.0,0.0
64172,2275,0.0,0.0,0.0,0.0
64223,2576,0.0,0.0,0.0,0.0


In [180]:
''' Quality of matches
    4.0: All 4 Records matches across all the four columns and 
    0.0: Zero matches (distinct record))'''

features.sum(axis=1).value_counts().sort_index(ascending=False)

4.0     818
3.0    1189
2.5      49
2.0    1087
1.5     145
1.0    1966
0.5     250
0.0    3953
dtype: int64

### Step 3: Final Mapping

In [ ]:
# Creating a score column and sorting the values by DBLP_Match

matched_results = features[features.sum(axis=1) > 2].reset_index()
matched_results['score'] = matched_results.loc[:, 'title':'year'].sum(axis=1)
matched_results.sort_values(by=['DBLP_Match'])

,Scholar_Match,DBLP_Match,title,authors,venue,year,score
1582,46547,1,1.0,1.0,0.0,1.0,3.0
1989,63690,2,1.0,1.0,1.0,1.0,4.0
843,21171,3,1.0,1.0,0.0,1.0,3.0
1375,38991,7,1.0,1.0,0.0,1.0,3.0
1422,40746,8,1.0,1.0,1.0,1.0,4.0
...,...,...,...,...,...,...,...
1238,35093,2608,1.0,1.0,1.0,1.0,4.0
442,8749,2610,1.0,1.0,0.0,1.0,3.0
1884,59086,2612,1.0,1.0,1.0,1.0,4.0
1088,29923,2614,1.0,1.0,0.0,1.0,3.0


Comment: A thresold of score above 2 is chosen to filter the final matches, which are 2056 matches (with 2.5 or above). Based on the requirements, a suitable of score thresold could be selected. 

In [ ]:
# Randomly verifying the outcome for similarity

print(df1.loc[46547,:])

print('\n*******************************************************************\n')

print(df2.loc[1,:])

idScholar                                         T2fm7Wb1ak4J
title        towards a cooperative transaction model the co...
authors          m rusinkiewicz w klas t tesch j waesch p muth
venue        proceedings of the international conference on...
year                                                      1995
Name: 46547, dtype: object

*******************************************************************

idDBLP                          conf/vldb/RusinkiewiczKTWM95
title      towards a cooperative transaction model the co...
authors         m rusinkiewicz w klas t tesch j w sch p muth
venue                                                   vldb
year                                                    1995
Name: 1, dtype: object


In [ ]:
# Creating a dataframe with Scholar_Match/ idScholar and DBLP_Match/ idDBLP

df1_lookup = df1['idScholar'].reset_index()

df2_lookup = df2['idDBLP'].reset_index()

df2_lookup

,DBLP_Match,idDBLP
0,1,conf/vldb/RusinkiewiczKTWM95
1,2,journals/sigmod/EisenbergM02
2,3,conf/vldb/AmmannJR95
3,4,journals/sigmod/Liu02
4,5,journals/sigmod/Hammer02
...,...,...
2611,2612,conf/vldb/ShuklaDNR96
2612,2613,journals/sigmod/Aberer03
2613,2614,conf/vldb/RamakrishnanR96
2614,2615,conf/vldb/ShaferAM96


In [ ]:
# Merging the results and creating a new column 'Match_ID' based on Scholar_match and DBLP_Match

df1_merge = matched_results.merge(account_lookup, how='left')

df2_merge = df1_merge.merge(df2_lookup, how='left')

df2_merge['Match_ID'] = df2_merge[['DBLP_Match','Scholar_Match']].astype(str).apply(lambda x: '_'.join(x), axis=1)

In [ ]:
df2_merge.sort_values(by=['DBLP_Match'])

,Scholar_Match,DBLP_Match,title,authors,venue,year,score,idScholar,idDBLP,Match_ID
1582,46547,1,1.0,1.0,0.0,1.0,3.0,T2fm7Wb1ak4J,conf/vldb/RusinkiewiczKTWM95,1_46547
1989,63690,2,1.0,1.0,1.0,1.0,4.0,wgK6p4mDSIMJ,journals/sigmod/EisenbergM02,2_63690
843,21171,3,1.0,1.0,0.0,1.0,3.0,x-H7BqZ0Hw8J,conf/vldb/AmmannJR95,3_21171
1375,38991,7,1.0,1.0,0.0,1.0,3.0,gEFY87Ma0XUJ,conf/vldb/SubietaKL95,7_38991
1422,40746,8,1.0,1.0,1.0,1.0,4.0,QTzV3iNq2O8J,journals/sigmod/BargaL02,8_40746
...,...,...,...,...,...,...,...,...,...,...
1238,35093,2608,1.0,1.0,1.0,1.0,4.0,RaOQG4YyzM4J,journals/sigmod/BusslerFM02,2608_35093
442,8749,2610,1.0,1.0,0.0,1.0,3.0,AfM_N94rVD0J,conf/vldb/Dyreson96,2610_8749
1884,59086,2612,1.0,1.0,1.0,1.0,4.0,vnezjPI8jYIJ,conf/vldb/ShuklaDNR96,2612_59086
1088,29923,2614,1.0,1.0,0.0,1.0,3.0,3y1e-H-FdC0J,conf/vldb/RamakrishnanR96,2614_29923


In [ ]:
# Creating a final mapping dataframe

column_names= ['idDBLP', 'idScholar', 'DBLP_Match', 'Scholar_Match', 'Match_ID']

final_mapping = df2_merge.reindex(columns=column_names)

final_mapping = final_mapping.sort_values(by=['DBLP_Match'])

final_mapping

,idDBLP,idScholar,DBLP_Match,Scholar_Match,Match_ID
1582,conf/vldb/RusinkiewiczKTWM95,T2fm7Wb1ak4J,1,46547,1_46547
1989,journals/sigmod/EisenbergM02,wgK6p4mDSIMJ,2,63690,2_63690
843,conf/vldb/AmmannJR95,x-H7BqZ0Hw8J,3,21171,3_21171
1375,conf/vldb/SubietaKL95,gEFY87Ma0XUJ,7,38991,7_38991
1422,journals/sigmod/BargaL02,QTzV3iNq2O8J,8,40746,8_40746
...,...,...,...,...,...
1238,journals/sigmod/BusslerFM02,RaOQG4YyzM4J,2608,35093,2608_35093
442,conf/vldb/Dyreson96,AfM_N94rVD0J,2610,8749,2610_8749
1884,conf/vldb/ShuklaDNR96,vnezjPI8jYIJ,2612,59086,2612_59086
1088,conf/vldb/RamakrishnanR96,3y1e-H-FdC0J,2614,29923,2614_29923


In [ ]:
# Storing the output into csv file

final_mapping.to_csv('DBLP_Scholar_perfectMapping_RaviLachireddy.csv', index=False)
